# Data preperation for the map visualisation #

Three dataset was used to create the map.
* [UK geoJson file](https://opendata.arcgis.com/datasets/fab4feab211c4899b602ecfbfbc420a3_4.geojson)
* [Accident data](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales)
* [Population Data](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland)

All datasets can be found in the folder called map_data together with the data generated through this code.

The process below consists of six parts:
* Matching accident data and geojson polygon
* Extracting data for total accidents
* Extracting data for speed
* Extracting data for casualties
* Extracting data for weather condition
* Extracting data for weekdays
* Cleaning population dataset
* Combining accident data with population data

# Match accident data and geojson polygon for 900.000 accidents #


### Function to generate data related to geoJson files ###
The function ckecks if the accident is within a given district. It the accident is, then it is assigned a district name corresponding to the goejson file.
The function ran for approx. 50 hours to check location of 900.000 accidents.

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import os
cwd = os.getcwd()

In [ ]:
# algorithm is based on: http://archived.mhermans.net/geojson-shapely-geocoding.html
from shapely.geometry import shape, Point

my_df = []

# load GeoJSON file containing sectors
with open(cwd + '/map_data/Local_Authority_Districts_Dec_2016.geojson') as f:
    js = json.load(f)

with open(cwd + '/map_data/CleanData_UK_Accidents.csv') as file:
    small_sample = pd.read_csv(file)
    
for i, row in small_sample.iterrows():
    # construct point based on lon/lat returned by geocoder
    point = Point(row.Longitude, row.Latitude)
    
    # check each polygon to see if it contains the point
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):

            d = {
                'lat': row.Latitude,
                'lon': row.Longitude,
                'name' : feature['properties']['lad16nm'],  # adding the district name
                'Number_of_Casualties' : row.Number_of_Casualties,
                'Road_Surface_Conditions' : row.Road_Surface_Conditions,
                'Weather_Conditions': row.Weather_Conditions,
                'Road_Type': row.Road_Type,
                'Speed_limit' : row.Speed_limit,
                'Date' : row.Date,
                'Day_of_Week' : row.Day_of_Week,
                'Urban_or_Rural_Area' : row.Urban_or_Rural_Area,
                'Accident_Severity' : row.Accident_Severity,
                'Junction_Control' : row.Junction_Control,
                'Light_Conditions' : row.Light_Conditions,
                'Year': row.Year
            }
            
            my_df.append(d)

print('Done')

csv_cityname = pd.DataFrame(my_df)

csv_cityname.to_csv('self_generated_data.csv')

# Exstracting data for total accients ##
The data is grouped and counted to show rank of accidents by the different distrcts in the bar chart.

In [475]:
sample = pd.read_csv(cwd + "/map_data/self_generated_data.csv");

In [476]:
sample = sample.groupby(['Year', 'name']).count()

In [477]:
sample = sample.rename(columns={'Unnamed: 0': 'accident_num'})

In [478]:
sample_to_export = sample['accident_num'].reset_index()

In [479]:
sample_to_export.head()

,Year,name,accident_num
0,2009,Aberdeen City,441
1,2009,Aberdeenshire,688
2,2009,Adur,157
3,2009,Allerdale,298
4,2009,Amber Valley,336


In [481]:
sample_to_export.to_csv('map_data/total_accidents_on_years_for_map.csv')

# Exstracting data for speed # 
Data is grouped and counted for the speed bar chart below the map

In [483]:
speed_df = pd.read_csv(cwd + "/map_data/self_generated_data.csv");

In [484]:
speed_df = speed_df.groupby(['Year', 'name','Speed_limit' ]).count()

In [485]:
speed_df = speed_df.rename(columns={'Unnamed: 0': 'accident_num'})

In [486]:
speed_df_to_export = speed_df['accident_num'].reset_index()

In [487]:
speed_df_to_export.head()

,Year,name,Speed_limit,accident_num
0,2009,Aberdeen City,20,44
1,2009,Aberdeen City,30,275
2,2009,Aberdeen City,40,48
3,2009,Aberdeen City,50,16
4,2009,Aberdeen City,60,36


In [489]:
speed_df_to_export.to_csv('map_data/total_accidents_on_years_and_speed.csv')

# Exstracting data for casualties # 
Data is grouped and counted for casualties bar chart below the map

In [490]:
casualties_df = pd.read_csv(cwd + "/map_data/self_generated_data.csv");

In [491]:
casualties_df = casualties_df.groupby(['Year', 'name','Number_of_Casualties' ]).count()

In [492]:
casualties_df = casualties_df.rename(columns={'Unnamed: 0': 'accident_num'})

In [493]:
casualties_df_to_export = casualties_df['accident_num'].reset_index()

In [494]:
casualties_df_to_export.head()

,Year,name,Number_of_Casualties,accident_num
0,2009,Aberdeen City,1,400
1,2009,Aberdeen City,2,33
2,2009,Aberdeen City,3,6
3,2009,Aberdeen City,4,1
4,2009,Aberdeen City,5,1


In [495]:
casualties_df_to_export.to_csv('map_data/total_accidents_on_years_and_casualties.csv')

# Exstracting data for weather conditions #
Data is grouped and counted for weather bar chart below the map

In [496]:
weather_df = pd.read_csv(cwd + "/map_data/self_generated_data.csv");

In [497]:
weather_df = weather_df.groupby(['Year', 'name','Weather_Conditions' ]).count()

In [498]:
weather_df = weather_df.rename(columns={'Unnamed: 0': 'accident_num'})

In [499]:
weather_df_to_export = weather_df['accident_num'].reset_index()

In [500]:
weather_df_to_export.head()

,Year,name,Weather_Conditions,accident_num
0,2009,Aberdeen City,Fine with high winds,10
1,2009,Aberdeen City,Fine without high winds,337
2,2009,Aberdeen City,Other,6
3,2009,Aberdeen City,Raining with high winds,15
4,2009,Aberdeen City,Raining without high winds,67


In [501]:
weather_df_to_export.to_csv('map_data/total_accidents_on_years_and_weather.csv')

# Exstracting data for weekdays # 
Data is grouped and counted for weekdays bar chart below the map

In [502]:
day_of_week_df = pd.read_csv(cwd + "/map_data/self_generated_data.csv");

In [503]:
day_of_week_df = day_of_week_df.groupby(['Year', 'name', 'Day_of_Week']).count()

In [504]:
day_of_week_df = day_of_week_df.rename(columns={'Unnamed: 0': 'accident_num'})

In [505]:
day_of_week_df_to_export = day_of_week_df['accident_num'].reset_index()

In [506]:
day_of_week_df_to_export.head()

,Year,name,Day_of_Week,accident_num
0,2009,Aberdeen City,1,46
1,2009,Aberdeen City,2,58
2,2009,Aberdeen City,3,72
3,2009,Aberdeen City,4,81
4,2009,Aberdeen City,5,71


In [507]:
day_of_week_df_to_export.to_csv('map_data/total_accidents_on_years_and_weekday.csv')

# Cleaning Population dataset# 
The population dataset is ceaned for columns that we do not need and the coulms are changed so it fits the structure we need.
The population dataset can be found here: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland


In [508]:
import pandas as pd
from pandas import *
pop_data = pd.read_csv(cwd + "/map_data/MYEB1_detailed_population_estimates_series_UK_2014.csv");

In [523]:
# selecting the columns that we need
pop_data = pop_data[[ 'lad2014_code', 'lad2014_name', 'population_2009', 'population_2010', 'population_2011', 'population_2012', 'population_2013', 'population_2014']]

In [511]:
pop_data.head()

,lad2014_code,lad2014_name,population_2009,population_2010,population_2011,population_2012,population_2013,population_2014
0,E06000001,Hartlepool,572,584,601,594,576,551
1,E06000001,Hartlepool,597,577,573,615,601,567
2,E06000001,Hartlepool,584,591,580,586,617,600
3,E06000001,Hartlepool,560,593,600,577,581,621
4,E06000001,Hartlepool,525,553,600,614,591,565


In [512]:
# group and summing the data
df = pop_data.groupby(["lad2014_code", "lad2014_name"])['population_2009', 'population_2010', 'population_2011', 'population_2012', 'population_2013', 'population_2014'].sum()

In [513]:
# renaming columns
df = df.rename(columns={'population_2009': '2009', 'population_2010': '2010', 'population_2011': '2011', 'population_2012' : '2012', 'population_2013' : '2013', 'population_2014' : '2014'})

In [514]:
df = df.reset_index()

In [515]:
# turning the columns into rows
df = df.melt(['lad2014_code', "lad2014_name"], var_name='2009', value_name='Val')

In [516]:
# renaming columns
df = df.rename(columns={'2009':'year', 'Val': 'population'})

In [517]:
# exporting the dataframe
df.to_csv("map_data/population_data_year_column.csv")

# Combining the accident data with the population data#
Since our self generated data was missing a column, we first mereged the original accident dataset with our dataset containing the district names and then created a column for normalized data. 

In [518]:
accident_df = pd.read_csv(cwd + "/map_data/total_accidents_on_years_for_map.csv");
population_df = pd.read_csv(cwd + "/map_data/population_data_year_column.csv");

In [519]:
# renaming the columns so that data can be merged
accident_df = accident_df.rename(columns={'Year' : 'year'})
population_df = population_df.rename(columns={'lad2014_name' : 'name'})

In [520]:
# the two datasets are merged on year and name to give us an population column
population_accident_df = pd.merge(accident_df, population_df, on=['year', 'name'], how='outer' )

In [521]:
# selecting the relevant columns
population_accident_df = population_accident_df[['year', 'name','accident_num', 'population']]

In [468]:
# creating a column for normalized data
population_accident_df['normalized'] = population_accident_df.accident_num/population_accident_df.population

In [522]:
population_accident_df.to_csv('map_data/population_and_accident_data_combined.csv')